In [ ]:
from openai import OpenAI
client = OpenAI(
    base_url="http://localhost:8000/v1"
)

completion = client.chat.completions.create(
    model="qwen",
    messages=[
        {"role": "user", "content": "Tell me a short story about king and queen"}
    ]
)

print(completion.choices[0].message)

In [ ]:
from openai import OpenAI
client = OpenAI(
    base_url="http://localhost:8000/v1"
)

completion = client.chat.completions.create(
    model="llama",
    messages=[
        {"role": "user", "content": "Answer with just city name. What i capital of india? "}
    ]
)

print(completion.choices[0].message)

In [ ]:
import json
import random
from openai import OpenAI

# Load ShareGPT data
with open("//home/skamalj/dev/vllm/ShareGPT_V3_unfiltered_cleaned_split.json", "r") as f:
    data = json.load(f)

# Flatten and filter valid user-only prompts
def extract_user_prompts(convo):
    return [msg["value"] for msg in convo["conversations"] if msg["from"] == "human"]

# Sample 100 random prompts
random_samples = random.sample(data, 25)

# Prepare OpenAI client (local server)
client = OpenAI(base_url="http://localhost:8000/v1")

# Iterate over samples
for idx, sample in enumerate(random_samples):
    prompts = extract_user_prompts(sample)
    if not prompts:
        continue

    # Take only the **first** user prompt per sample
    first_prompt = prompts[0].strip()
    print(f"Processing sample #{idx + 1} with prompt: {first_prompt}")

    try:
        completion = client.chat.completions.create(
            model="my-mistral-model",
            messages=[{"role": "user", "content": first_prompt}]
        )

        print(f"Sample #{idx + 1}")
        print("Prompt:", first_prompt)
        print("Response:", completion.choices[0].message.content.strip())
        print("-" * 50)

    except Exception as e:
        print(f"Error on sample #{idx + 1}: {e}")


> Download  data file from [here](https://huggingface.co/datasets/anon8231489123/ShareGPT_Vicuna_unfiltered)

In [4]:
import json
import random
from openai import OpenAI
from concurrent.futures import ThreadPoolExecutor, as_completed

# Load ShareGPT data
with open("/home/skamalj/dev/hfdata/ShareGPT_V3_unfiltered_cleaned_split.json", "r") as f:
    data = json.load(f)

# Flatten and filter valid user-only prompts
def extract_user_prompts(convo):
    return [msg["value"] for msg in convo["conversations"] if msg["from"] == "human"]

# Sample 100 random prompts
random_samples = random.sample(data, 10)
client = OpenAI(base_url="http://localhost:8000/v1")

# Prepare request payloads
requests = []
for sample in random_samples:
    prompts = extract_user_prompts(sample)
    if prompts:
        requests.append(prompts[0].strip())

# Function to run a single request
def run_request(idx, prompt):
    try:
        completion = client.chat.completions.create(
            model="llama",
            messages=[{"role": "user", "content": prompt}]
        )
        return idx, prompt, completion.choices[0].message.content.strip()
    except Exception as e:
        return idx, prompt, f"ERROR: {e}"

# Run in parallel to trigger batching in vLLM
with ThreadPoolExecutor(max_workers=5) as executor:
    futures = [executor.submit(run_request, idx, prompt) for idx, prompt in enumerate(requests)]
    for future in as_completed(futures):
        idx, prompt, response = future.result()
        print(f"Sample #{idx + 1}")
        print("Prompt:", prompt)
        print("Response:", response)
        print("-" * 50)


Sample #4
Prompt: I am still only getting the same index data when I press view film details
Response: ERROR: Connection error.
--------------------------------------------------
Sample #1
Prompt: can you continue the conversation until a novel, promising solution is identified?
Response: ERROR: Connection error.
--------------------------------------------------
Sample #2
Prompt: 0 degree: <45 = 10/10, >45 = 0/10 

15 degree: <45 = 6/10, >45 = 4/10 

30 degree: <45 = 10/10, >45 = 0/10 

45 degree: <45 = 9/10, >45 = 1/10 

60 degree: <45 = 3/10, >45 = 7/10 

75 degree: <45 = 2/9, >45 = 7/9 
90 degree: <45 = 0/9, >45 = 9/9
Response: ERROR: Connection error.
--------------------------------------------------
Sample #3
Prompt: Could Enrile be either a lobster or a jellyfish? What's more plausible between the two?
Response: ERROR: Connection error.
--------------------------------------------------
Sample #5
Prompt: when is Ramadan 2023?
Response: ERROR: Connection error.
-----------------